In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit

In [2]:
def get_bits(x, n):
    return [int(x) for x in '{:0{size}b}'.format(x, size=n)]

In [3]:
def generate_carry(constant, n):
    name = 'Carry ({0})'.format(constant) # <---
    constant_bits = get_bits(constant, n)
    
    qrCtrl = QuantumRegister(1, name='ctrl')
    qrA = QuantumRegister(n, name='a') # |a> - input register
    qrC = QuantumRegister(1, name='c') # |c> - output register - information about carry
    
    # special case for n = 1: 
    if n == 1:
        qc = QuantumCircuit(qrCtrl, qrA, qrC, name=name) # <---
    
        if constant_bits[0] == 1:
            qc.ccx(qrCtrl[0], qrA[0], qrC)
        
        return qc.to_instruction() # <---
    
    # for n >= 2:
    qrG = QuantumRegister(n-1, name='g') # |g> - dirty ancilla register, state have to be restored
    
    qc = QuantumCircuit(qrCtrl, qrA, qrG, qrC, name=name) # <---

    qc.ccx(qrCtrl[0], qrG[n - 2], qrC)

    for i in range(n - 1, 1, -1):
        if constant_bits[n - (i + 1)] == 1:
            qc.cx(qrA[i], qrG[i - 1])
            qc.x(qrA[i])
        qc.ccx(qrG[i - 2], qrA[i], qrG[i - 1])
        
        # qc.barrier(qrA[:], qrG[:], qrC[:])

    if constant_bits[n - 2] == 1:
        qc.cx(qrA[1], qrG[0])
        qc.x(qrA[1])
        
    if constant_bits[n - 1] == 1:
        qc.ccx(qrA[0], qrA[1], qrG[0])

    for i in range(2, n):
        qc.ccx(qrG[i - 2], qrA[i], qrG[i - 1])

    # qc.barrier(qrA[:], qrG[:], qrC[:])

    qc.ccx(qrCtrl[0], qrG[n - 2], qrC)

    # qc.barrier(qrA[:], qrG[:], qrC[:])
   
    for i in range(n - 1, 1, -1):
        qc.ccx(qrG[i - 2], qrA[i], qrG[i - 1])

    if constant_bits[n - 1] == 1:
        qc.ccx(qrA[0], qrA[1], qrG[0])
        
    if constant_bits[n - 2] == 1:
        qc.x(qrA[1])
        qc.cx(qrA[1], qrG[0])

    for i in range(2, n):
        # qc.barrier(qrA[:], qrG[:], qrC[:])
        qc.ccx(qrG[i - 2], qrA[i], qrG[i - 1])
        if constant_bits[n - (i + 1)] == 1:
            qc.x(qrA[i])
            qc.cx(qrA[i], qrG[i - 1])
     
    # qc.barrier(qrA[:], qrG[:], qrC[:])
    
    return qc.to_instruction() # <---

In [4]:
# _qrCtrl = QuantumRegister(1, name='ctrl')
# _qrA = QuantumRegister(4, name='a')
# _qrG = QuantumRegister(3, name='g')
# _qrC = QuantumRegister(1, name='c')

# _qc = QuantumCircuit(_qrCtrl, _qrA, _qrG, _qrC)

# carry = generate_carry(11, 4)

# _qc.append(carry, _qrCtrl[:] + _qrA[:] + _qrG[:] + _qrC[:])
# _qc.draw(output='mpl')

In [5]:
# _dec = _qc.decompose()
# _dec.draw(output='mpl')

In [6]:
def generate_init_part(qc, ctrl, a, g, n):
    a_bits = get_bits(a, n)
    
    if n == 1:
        [qrCtrl, qrA, qrC] = qc.qregs
    else:
        [qrCtrl, qrA, qrG, qrC] = qc.qregs
    
    if ctrl:
        qc.x(qrCtrl[0])
    
    if n == 1:        
        if a_bits[0] == 1:
            qc.x(qrA[0])
        
        qc.barrier(qrA[:])
        return
        
    g_bits = get_bits(g, n - 1)
    
    for i in range(n):
        if a_bits[n - (i+1)] == 1:
            qc.x(qrA[i])

    for i in range(n - 1):
        if g_bits[n-1 - (i+1)] == 1:
            qc.x(qrG[i])

    qc.barrier(qrA[:], qrG[:])

In [7]:
# _qrCtrl = QuantumRegister(1, name='ctrl')
# _qrA = QuantumRegister(4, name='a')
# _qrG = QuantumRegister(3, name='g')
# _qrC = QuantumRegister(1, name='c')

# _qc = QuantumCircuit(_qrCtrl, _qrA, _qrG, _qrC)

# generate_init_part(_qc, True, 6, 5, 4)

# carry = generate_carry(6, 4)
# _qc.append(carry, _qrCtrl[:] + _qrA[:] + _qrG[:] + _qrC[:])

# _qc.draw(output='mpl')

In [8]:
def generate_measure_part(n):
    qrA = QuantumRegister(n, name='a')
    qrC = QuantumRegister(1, name='c')
    crA = ClassicalRegister(n, name='aValue')
    crC = ClassicalRegister(1, name='cValue')
    
    if n == 1:
        qc = QuantumCircuit(qrA, qrC, crA, crC)
        qc.barrier(qrA[:], qrC[:])
        qc.measure(qrA[:], crA[:])
        qc.measure(qrC[:], crC[:])
        
        return qc
    
    qrG = QuantumRegister(n - 1, name='g')
    crG = ClassicalRegister(n - 1, name='gValue')
    
    qc = QuantumCircuit(qrA, qrG, qrC, crA, crG, crC)
    qc.barrier(qrA[:], qrG[:], qrC[:])
    qc.measure(qrA[:], crA[:])
    qc.measure(qrG[:], crG[:])
    qc.measure(qrC[:], crC[:])
    
    return qc

In [9]:
# qc_meas = generate_measure_part(4)
# qc_meas.draw(output='mpl')

In [10]:
def test_carry(ctrl, a, constant, n, carry):
    expected_carry = int((a + constant) / (2 ** n)) if ctrl else 0
    print(expected_carry, carry, expected_carry == carry)

In [11]:
# test_carry(6, 11, 4, 1)
# test_carry(True, 6, 11, 4, 1)
# test_carry(False, 6, 11, 4, 0)

In [12]:
from qiskit import Aer
from qiskit import execute

backend = Aer.get_backend('qasm_simulator')

In [13]:
# a = 6
# g = 5
# constant = 11
# n = 4

In [14]:
def test(ctrl, a, g, constant, n):
    qrCtrl = QuantumRegister(1, name='ctrl')
    qrA = QuantumRegister(n, name='a') 
    qrC = QuantumRegister(1, name='c')
    
    if n == 1:
        qc = QuantumCircuit(qrCtrl, qrA, qrC)
    else:
        qrG = QuantumRegister(n - 1, name='g')
        qc = QuantumCircuit(qrCtrl, qrA, qrG, qrC)        

    generate_init_part(qc, ctrl, a, g, n)

    carry = generate_carry(constant, n)
    if n == 1:
        qc.append(carry, qrCtrl[:] + qrA[:] + qrC[:])
    else:
        qc.append(carry, qrCtrl[:] + qrA[:] + qrG[:] + qrC[:])
         
    qc += generate_measure_part(n)

    job = execute(qc, backend, shots=1)
    result = job.result()
    values = list(result.get_counts(qc).keys())

    assert len(values) == 1
    values = [int(v, 2) for v in values[0].split(' ')]

    if n == 1:
        c_value = values[0]
        a_value = values[1]

        print(a, a_value, a == a_value)
        test_carry(ctrl, a, constant, n, c_value)
    else:
        c_value = values[0]
        g_value = values[1]
        a_value = values[2]

        print(a, a_value, a == a_value)
        print(g, g_value, g == g_value)
        test_carry(ctrl, a, constant, n, c_value)

In [15]:
test(True, 6, 5, 11, 4)
test(False, 6, 5, 11, 4)

<ipython-input-14-cdf0f17c73ab>:20: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  qc += generate_measure_part(n)
/home/bartek/MGR/masterThesisProject/venv/lib/python3.8/site-packages/qiskit/circuit/quantumcircuit.py:876: DeprecationWarning: The QuantumCircuit.extend() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  return self.extend(rhs)


6 6 True
5 5 True
1 1 True
6 6 True
5 5 True
0 0 True


In [16]:
# for n in range(4):
#     n = n + 1
#     G = 2 ** (n-1)
#     N = 2 ** n
#     for a in range(N):
#         for g in range(G):
#             for c in range(N):
#                 for ctrl in range(2):
#                     print('---', ctrl, '-', a, g, c, '---')
#                     test(ctrl == 1, a, g, c, n)